In [60]:
import numpy as np
from tqdm import tqdm
import os
import json
from torch.utils.data import Dataset, Sampler
import torch.utils.data as data

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets,models,transforms
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import math
import os
import copy
import time
import sqlite3 as sql
import cv2
from tqdm import tqdm

/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


### Get features, labels and timecodes

In [53]:
features =[]
labels = []
timeLabel = []

path = '/scratch/users/mdelabrassinne/Database/'


for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    for file in files:
            
        if file.endswith("Labels-v2.json"):
            
            p = os.path.dirname(os.path.join(root, file))
            dire = os.path.basename(p).replace('_', ' ')
            p = os.path.join(os.path.dirname(p), dire)
            
            for r, d, fs in os.walk(p):
                for f in fs:
                    if f.endswith("baidu_soccer_embeddings.npy"):
                        p_current = os.path.join(r, f)
                        features.append(p_current)
                        #print(p_current)

            
            jsonPath =(os.path.join(root, file))
            #print(jsonPath)
            f = open(jsonPath)
            d = json.load(f)
            #print(d)
            
            #print(jsonPath)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']
                #print([d['annotations'][i]['gameTime'] for i in range(len(d['annotations']))])
                
                #print(time)
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']
                
                #print(time)
                labels.append(lab)
                timeLabel.append(time)
                
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']
                
                labels.append(lab)
                timeLabel.append(time)
            

            
data_l = list(zip(features, labels, timeLabel))

98714it [01:02, 1578.38it/s]


### Function for the transformation of the ground truths

In [5]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

def getTrueCoef(label, timeCodes, T, nbClasses, r_c = 3, f = 1, r_d = 6):
    C = []      #confidence as a 2D np array instead of a list of tupes

    res_conf = np.zeros((T, nbClasses))
    res_disp = np.zeros((T, nbClasses))
    
    dictClasses = {}

    for c in range(nbClasses):
        dictClasses[c] = []


    for i in range(len(timeCodes)):        #loop over obtained labels
        t = timeCodes[i]
        c = label[i]
        
        if(c in cL):                           # focus on wanted classes
            c = cL.index(label[i])

            tmin = max(0, t - r_c * f)
            tmax = min(t + r_c * f, T-1)
            inds = np.arange(tmin, tmax+1)
            res_conf[inds, c] = 1
            
            dictClasses[c].append(t)
            
        if(t > T):
            print('\n\naie => t = {} -- T = {}\n\n'.format(t, T))
            
        if(i >= 2700):
            print("possible issue ")

    for c in range(nbClasses):
        ind = np.array(dictClasses[c])
        if(len(ind) > 0):
            for t in range(T):

                try:
                    closestInd = np.argmin(np.abs(ind - t))         #take the one that is the most on the left if draw
                except:
                    print(ind)
                    print(t)
                    print(len(ind))

                if(np.abs(t - ind[closestInd]) <= r_d * f):
                    res_disp[t, c] = t - ind[closestInd]
            
        
    return res_conf, res_disp

test_C, test_D = getTrueCoef(data_l[0][1],data_l[0][2], np.load(data_l[0][0]).shape[0], len(cL) )

print(np.where(test_C != 0))
print(np.where(test_D != 0))

print(test_C.shape)
print(test_D.shape)

(array([   0,    1,    2,    3,  179,  180,  181,  182,  183,  184,  185,
        289,  290,  291,  292,  293,  294,  295,  664,  665,  666,  667,
        668,  669,  670,  787,  788,  789,  790,  791,  792,  793,  840,
        841,  842,  843,  844,  845,  846,  877,  878,  879,  880,  881,
        882,  883,  968,  969,  970,  971,  972,  973,  974, 1028, 1029,
       1030, 1031, 1032, 1033, 1034, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2542, 2543, 2544,
       2545, 2546, 2547, 2548]), array([3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0]))
(array([   1,    2,    3,    4,    5,    6,  176,  177,  178,  179,  180,
        181,  183,  184,  185,  186,  187,  188,  286,  287,  288,  289,
        290,  291,  293,  294,  295

### Dataset, dataloader

In [8]:
print(data_l)

[('/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy', ['Kick-off', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Throw-in', 'Shots on target', 'Ball out of play', 'Corner', 'Offside', 'Shots on target', 'Offside', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Shots off target', 'Ball out of play', 'Clearance', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Clearance', 'Ball out of play', 'Goal', 'Kick-off', 'Shots on target', 'Ball out of play', 'Corner', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Corner', 'Shots off target', 'Clearance', 'Ball out of play', 'Clearance', 'Ball out of play', 'Clearance', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Shots on target', 'Foul', 'Shots off target',

In [56]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes

        features = np.load(feat_path)
        
        indStart = np.ceil(np.random.uniform(0, features.shape[0]-self.seq_length))
        ind = np.array([True if (i >= indStart and i < indStart + self.seq_length) else False for i in range(features.shape[0])])
        
        coef_c, coef_d = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )
        
        print(features[ind,:].shape)
        print(coef_c[ind,:].shape)
        print(coef_d[ind,:].shape)
        print("")
        
        #print("\n")
        return (features[ind,:], coef_c[ind, :], coef_d[ind, :], idx)   

### Test dataloader

In [58]:
dataLearning = BaiduDataset(data_l)
loaderLearning = data.DataLoader(dataLearning, batch_size = 20, shuffle= True, num_workers = 1)
feat, lab, t, idx = next(iter(loaderLearning))

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 8576)
(112, 7)
(112, 7)

(112, 85

### Model

In [61]:
MLP_KERNEL_SIZE = 1
MLP_FILTERS_1 = 256
MLP_FILTERS_2 = 64
CONTRACTION_RATIO = 2
PADDING_METHOD = 'same'
UPSAMPLE_MODE = 'nearest'
N_FEATURE_EXTRACTED = 2048
N_CLASSES = 17
SEQUENCE_SIZE = 112
BATCH_SIZE = 256
BATCH_PER_EPOCH = 1000

In [66]:
class MLP2(nn.Module):
    def __init__(self,n_features):
        super(MLP2,self).__init__()
        self.conv1 = nn.Conv1d(n_features,MLP_FILTERS_1,MLP_KERNEL_SIZE)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(MLP_FILTERS_1,MLP_FILTERS_2,MLP_KERNEL_SIZE)

    def forward(self,x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out


In [63]:
class BottleneckResBlockContract(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P) ->BN ->ReLU -> conv(3,P) -> BN -> ReLU -> conv(1,2P) -> + -> output (of size T,2P)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,2P) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        super(BottleneckResBlockContract,self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,in_features,1)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.conv2 = nn.Conv1d(in_features,in_features,3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(in_features)
        self.conv3 = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)
        self.conv_skip = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z
    
class BottleneckResBlockExpand(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P/2) ->BN ->ReLU -> conv(3,P/2) -> BN -> ReLU -> conv(1,P/2) -> + -> output (of size T,P/2)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,P/2) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        in_features = int(in_features)
        super(BottleneckResBlockExpand,self).__init__()
        self.bn1 = nn.BatchNorm1d(int(in_features))
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)
        self.bn2 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv2 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv3 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),1)
        self.conv_skip = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z

class UpsamplingBlock(nn.Module):
# Takes as input a feature matrix of size (with ncol: nFeatures and nrow: nTimePoints) :
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
# Upsample uses nearest neighbor to output a 2TxP matrix
# a Conv(2,P/2) is then applied with padding = same
    
    def __init__(self,in_feature):
        super(UpsamplingBlock,self).__init__()
        in_feature = int(in_feature)
        self.upsample = nn.Upsample(scale_factor=2,mode=UPSAMPLE_MODE)
        self.conv = nn.Conv1d(in_feature, int(in_feature/2),kernel_size=2, padding=PADDING_METHOD)
    
    def forward(self,x):
        out = self.upsample(x)
        out = self.conv(out)
        return out
    

class Unet1D(nn.Module):
    def __init__(self,in_feature):
        super(Unet1D,self).__init__()

        #LEFT PART
        self.maxpool = nn.MaxPool1d(2)

        self.contr1 = BottleneckResBlockContract(in_feature)
        #self.maxpool
        nb_f = in_feature*2
        self.contr2 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr3 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr4 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        ##############################################

        #DOWN PART
        
        self.contr5 = BottleneckResBlockContract(nb_f)
        nb_f = nb_f*2
        ##############################################

        #RIGHT PART

        self.upsample4 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand4 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample3 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand3 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample2 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand2 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample1 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand1 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample0 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand0 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

    def forward(self,x):
        # X is of shape [nbatch,64,224]

        # LEFT PART
        l0 = x #[P,T]           #Output shape
        a = self.maxpool(l0)    #[P,T/2]
        l1 = self.contr1(a)     #[P*2,T/2]
        b = self.maxpool(l1)    #[P*2,T/4]
        l2 = self.contr2(b)     #[P*4,T/4]
        c = self.maxpool(l2)    #[P*4,T/8]
        l3 = self.contr3(c)     #[P*8,T/8]
        d = self.maxpool(l3)    #[P*8,T/16]
        l4 = self.contr4(d)     #[P*16,T/16]
        e = self.maxpool(l4)    #[P*16, T/32]

        # DOWN PART
        l5 = self.contr5(e)     #[P*32,T/32]

        # RIGHT PART
        r4 = self.upsample4(l5)     #[P*16,T/16]
        t4 = torch.cat((l4,r4),1)   #[P*32,T/16]
        t4 = self.expand4(t4)       #[P*16,T/16]
        r3 = self.upsample3(t4)     #[P*8,T/8]
        t3 = torch.cat((l3,r3),1)   #[P*16,T/8]
        t3 = self.expand3(t3)       #[P*8,T/8]
        r2 = self.upsample2(t3)     #[P*4,T/4]
        t2 = torch.cat((l2,r2),1)   #[P*8,T/4]
        t2 = self.expand2(t2)       #[P*4,T/4]
        r1 = self.upsample1(t2)     #[P*2,T/2]
        t1 = torch.cat((l1,r1),1)   #[P*4,T/2]
        t1 = self.expand1(t1)       #[P*2,T/2]
        r0 = self.upsample0(t1)     #[P,T]
        t0 = torch.concat((l0,r0),1)#[P*2,T]
        t0 = self.expand0(t0)       #[P,T]

        return t0 # P features, T points

In [64]:
class PredictionHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


In [65]:
class TemporalHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


### test loss

In [8]:
import torch.nn as nn

In [9]:
def huberLoss(pred, gTruth, treshold=0.5):
    res = pred - gTruth
    
    abs_res = np.abs(res)
    quadratic_loss = 0.5 * np.square(res)
    linear_loss = treshold * (abs_res - 0.5 * treshold)
    
    loss = np.where(abs_res <= treshold, quadratic_loss, linear_loss)
    return loss

In [10]:
def confLoss2(c_preds, c_truths, T):
    if(c_preds.shape[0] != T):
        c_preds = c_preds.reshape(-1)

    if(c_truths.shape[0] != T):
        c_truths = c_truths.reshape(-1)

    criterion = nn.CrossEntropyLoss()       

    return criterion(c_preds, c_truths)


def dispLoss2(d_preds, d_truths, T):
    if(d_preds.shape[0] != T):
        d_preds = d_preds.reshape(-1)

    if(d_truths.shape[0] != T):
        d_truths = d_truths.reshape(-1)

    return huberLoss(d_preds, d_truths)


def loss(c_preds, d_preds, c_truths, d_truths, T):
    l = confLoss2(c_preds, c_truths, T)
    

In [12]:
test_c = np.random.randn(2700, 7)
test_d = np.random.randn(2700, 7)

true_c = np.random.randn(2700, 7)
true_d = np.random.randn(2700, 7)

### Training

In [ ]:
def make():
    # Make the model
    mod = model.to(device)                                     #model variable globale

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(mod.parameters(), lr=learning_rate, weight_decay=decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold  = .01, threshold_mode = "abs", mode='max', factor=0.2, patience=5)
    
    return criterion, optimizer, scheduler

In [ ]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [ ]:
def test():
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        correct_class = [0 for i in range(n_classes)]
        total_class = [0 for i in range(n_classes)]
        for images, labels in loaderValidation:
            
            labels = toTens(labels)
            
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Compute accuracy per class
            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                total_class[label] += 1
                if label == pred:
                    correct_class[label] += 1
            torch.cuda.empty_cache()

        
        test_acc = 0.
        for i in range(n_classes):
            test_acc += correct_class[i] / total_class[i]
            wandb.log({f"Accuracy of {c[i]}": correct_class[i] / total_class[i]})
        test_acc /= n_classes
        
        print(f"Accuracy of the model on the {total} " +
              f"test images: {test_acc:%}")
        
        wandb.log({"test_accuracy": test_acc})

        
        
    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")
    return correct / total

In [ ]:
def train(criterion, optimizer, scheduler, test_set = True):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loaderLearning) * epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
        
    for epoch in tqdm(range(epochs)):
        for images, labels in loaderLearning:
            
            #images = images.to(device)
            labels = toTens(labels)
            
            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)
                
            torch.cuda.empty_cache()

        if test_set:
            acc = test()
            scheduler.step(acc)
            


def train_batch(images, labels, model, optimizer, criterion):
    
    images, labels = images.to(device), labels.to(device)
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [ ]:
config = dict(
    epochs=30,
    classes=6,
    learning_rate=0.0001,
    loaderLearning = loaderLearning,
    loaderValidation = loaderValidation,
    loaderTest = loaderTest)

In [ ]:
def model_pipeline(config=None, project_name="action_spotting"):

    # tell wandb to get started
    with wandb.init(config=config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # data, and optimization problem
      criterion, optimizer, scheduler  = make()  
      # and use them to train the model
      train(criterion, optimizer, scheduler)

    return model

In [ ]:
sweep_config = {
    "name": "action_spotting",
    "method": "random",
    "metric": {"name": "test_accuracy", "goal": "maximize"},
    "parameters":
    {
        "learning_rate": {"values": [0.01]},
        "batch_size": {"values": [100]},
    }
}
# sweep_id = wandb.sweep(sweep_config, project="action_spotting")

wandb.login()